## General TM routing with Pyxem. 

Parameters in this notebook are optimized for ErMnO3 1deg SPED 10 ms exposure time

In [1]:
%matplotlib qt

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import hyperspy.api as hs
from pathlib import Path
import os

In [3]:
folder = "datapath"

file = "SPED_600_280_correct_entire_lamella_10ms_Cl12_step20p8m_tx3p2_ty2p4_spot1nm_aplh5.hspy"
filepath = os.path.join(folder, file)
org_data = hs.load(filepath) 

In [5]:
org_data.metadata

├── Acquisition_instrument
│   ├── Merlin_hdr
│   │   ├── Acquisition Type (Normal, Th_scan, Config) = Normal
│   │   ├── Active Counters = Counter 0
│   │   ├── Assembly Size (NX1, 2X2) = 1x1
│   │   ├── Chip ID = W559_G11, - , - , -
│   │   ├── Chip Mode  (SPM, CSM, CM, CSCM) = SPM
│   │   ├── Chip Type (Medipix 3.0, Medipix 3.1, Medipix 3RX) = Medipix 3RX
│   │   ├── Counter Depth (number) = 12
│   │   ├── DAC File = c:\MERLIN_Quad_Config\W559_G11\W559_G11_SPM.dacs,,,
│   │   ├── DACs = 068,511,000,000,000,000,000,000,100,255,100,125,100,100,080,100,090,030,128,004,255,126,128,174,172,511,511; ; ;
│   │   ├── Dead Time File = Dummy (C:\<NUL>\)
│   │   ├── Flat Field File = None
│   │   ├── Frames in Acquisition (Number) = 321024
│   │   ├── Frames per Trigger (Number) = 1
│   │   ├── Gain = SLGM
│   │   ├── Gap Fill Mode = None
│   │   ├── Humidity (%) = Board Humidity 0.000000
│   │   ├── Medipix Clock (MHz) = 160MHz
│   │   ├── Readout System = Merlin Quad
│   │   ├── Sensor Bias (V) = 120 V
│   │   ├── Sensor Polarity (Positive, Negative) = Positive
│   │   ├── Software Version = 1.0.0.122
│   │   ├── Temperature (C) = Board Temp 0.000000 Deg C
│   │   ├── Thresholds (keV) = 4.000000E+1,5.110000E+2,0.000000E+0,0.000000E+0,0.000000E+0,0.000000E+0,0.000000E+0,0.000000E+0
│   │   ├── Time and Date Stamp (day, mnth, yr, hr, min, s) = 20/02/2023 18:28:07
│   │   ├── Trigger Start (Positive, Negative, Internal) = Rising Edge
│   │   ├── Trigger Stop (Positive, Negative, Internal) = Internal
│   │   └── bpc File = c:\MERLIN_Quad_Config\W559_G11\W559_G11_SPM.bpc,,,
│   └── TEM
│       ├── Detector
│       │   └── Diffraction
│       │       ├── camera_length = 22.95
│       │       └── exposure_time = 10
│       ├── beam_energy = 200
│       ├── rocking_angle = 0
│       └── rocking_frequency = 100
├── General
│   ├── FileIO
│   │   ├── 0
│   │   │   ├── hyperspy_version = 1.7.1
│   │   │   ├── io_plugin = hyperspy.io_plugins.hspy
│   │   │   ├── operation = save
│   │   │   └── timestamp = 2023-02-21T09:21:21.303346+01:00
│   │   └── 1
│   │       ├── hyperspy_version = 1.7.1
│   │       ├── io_plugin = hyperspy.io_plugins.hspy
│   │       ├── operation = load
│   │       └── timestamp = 2023-02-21T15:45:57.305289+01:00
│   └── title = 
└── Signal
    ├── exposure_time = [0.005]
    ├── flyback_times = None
    ├── frames_number_skipped = None
    ├── scan_X = None
    └── signal_type = electron_diffraction

In [ ]:
work_data = org_data #Could do .inav[::2,::2] to decrease number of pixels and computation time
work_data.plot()

In [7]:
center_direct_beam_dict = {
    "method" : "blur",
    "half_square_width" : 10,
    "sigma" : 3
}
work_data.metadata.add_node("center_direct_beam_kwargs")
work_data.metadata.center_direct_beam_kwargs =  center_direct_beam_dict

In [8]:
work_data.center_direct_beam(**work_data.metadata.center_direct_beam_kwargs)

[########################################] | 100% Completed | 92.62 s
[########################################] | 100% Completed | 33.79 s


In [9]:
work_data.plot(vmax = 50, cmap = 'viridis')

In [10]:
from pyxem.utils.expt_utils import investigate_dog_background_removal_interactive

In [12]:
gauss_stddev_maxs = np.arange(2, 12, 0.2) # min, max, step
gauss_stddev_mins = np.arange(1, 4, 0.2) # min, max, step


dp_test_area = work_data.inav[150,150]
investigate_dog_background_removal_interactive(dp_test_area,
                                               gauss_stddev_maxs,
                                               gauss_stddev_mins)

/home/anderscm/miniconda3/envs/pyxem142/lib/python3.9/site-packages/hyperspy/drawing/image.py:561: UserWarning: Attempted to set non-positive bottom ylim on a log-scaled axis.
Invalid limit will be ignored.
  ims[0].set_norm(norm)


In [13]:
diff_background_removal_dict = {
    "method" : "difference of gaussians",
    "min_sigma":3,
    "max_sigma":6
}
work_data.metadata.add_node("subract_diff_background_kwargs")
work_data.metadata.subract_diff_background_kwargs = diff_background_removal_dict

In [14]:
#test preprocessing on a single PED pattern
# If looks OK, parameters are used for entire SPED dataset

testimg = work_data.inav[250,250].subtract_diffraction_background(**work_data.metadata.subract_diff_background_kwargs )
testimg.plot( norm = 'symlog')

In [15]:
from skimage import filters

In [16]:
def threshold_minimum(image, minimum=0.1):
    image[image <= minimum] = 0.
    return image

In [17]:

testimg.map(filters.gaussian, sigma=0.5)
testimg.map(threshold_minimum, minimum = 1)
testimg.map(filters.gaussian, sigma=0.5)
testimg.map(filters.gaussian, sigma = 1)
testimg.map(threshold_minimum, minimum = 0.3)

In [18]:
testimg.plot( norm = 'symlog')

In [19]:
#Preprocessing on entire SPED data

work_data = work_data.subtract_diffraction_background(**work_data.metadata.subract_diff_background_kwargs,)
                                            
work_data.map(filters.gaussian, sigma=0.5)
work_data.map(threshold_minimum, minimum = 1)
work_data.map(filters.gaussian, sigma=0.5)
work_data.map(filters.gaussian, sigma = 1)
work_data.map(threshold_minimum, minimum = 0.3)

In [21]:
work_data.plot(norm = 'symlog', cmap = 'inferno_r')

[########################################] | 100% Completed | 464.64 s


In [23]:
#Make diffraction library, loads file if diff library already excists

from Utils.GetDiffLibrary import GetDiffLibrary

diffraction_calibration = 0.00952354965
half_radius = 80
diff_lib = GetDiffLibrary(diffraction_calibration, 
                        12,
                        half_radius,
                        resolution=0.3,
                        make_new=False,
                        grid_cub = None, 
                        minimum_intensity=0.0015,
                        max_excitation_error=0.0125,
                        precession_angle =0.3 
                        )

simulations = diff_lib["ErMnO3"]["simulations"]
orientations = diff_lib["ErMnO3"]["orientations"]

In [25]:

from pyxem.utils import indexation_utils as iutls
from pyxem.utils import plotting_utils as putls

In [26]:
#Select a PED pattern to test TM on before running TM on full dataset

image = work_data.inav[250,260].data

In [27]:
indexes, angles, corrs, angles_m, corrs_m = iutls.correlate_library_to_pattern(image, simulations)

In [28]:
from orix.quaternion import Orientation, symmetry
from orix.vector import Vector3d
from diffsims.generators.rotation_list_generators import get_beam_directions_grid
grid_cub = get_beam_directions_grid("hexagonal", 0.3, mesh="spherified_cube_edge")
origrid = Orientation.from_euler(
    np.radians(grid_cub),
    symmetry=symmetry.C6h

)
v = Vector3d(((0, 0, 1)))
origrid.scatter("ipf", direction=v, s = 10)

/home/anderscm/miniconda3/envs/pyxem142/lib/python3.9/site-packages/diffsims/generators/sphere_mesh_generators.py:515: RuntimeWarning: invalid value encountered in true_divide
  phi2 = sign * np.nan_to_num(np.arccos(x_comp / norm_proj))


In [29]:

all_cors = np.stack([corrs, corrs_m])
vmin = all_cors.min()
vmax = all_cors.max()

fig = plt.figure()
ax0 = fig.add_subplot(121, projection="ipf", direction=v, symmetry=symmetry.C6v)
ax1 = fig.add_subplot(122, projection="ipf", direction=v, symmetry=symmetry.C6v)
ax0.set_title("Correlations")
ax1.set_title("Mirrored correlations")
ax0.scatter(origrid[indexes], c = corrs, cmap = "inferno", vmin = vmin, vmax = vmax)
ax0.scatter(origrid[corrs.argmax()])
ax1.scatter(origrid[indexes], c = corrs_m, cmap = "inferno", vmin = vmin, vmax = vmax)

In [30]:
indices_n, angles_n, correlations_n, signs_n = iutls.get_n_best_matches(image, simulations,)

In [31]:
mirrored = signs_n[0] == -1
putls.plot_template_over_pattern(image,
                                 simulations[indices_n[0]],
                                 in_plane_angle=angles_n[0],
                                 size_factor = 1,
                                 vmax=20,
                                 mirrored_template=mirrored,
                                 find_direct_beam=False,
                                 cmap = "inferno"
                                )

(<AxesSubplot:>,
 <matplotlib.collections.PathCollection at 0x7f77bdb55c70>)

In [32]:
#introduce log scaling on image, which gives more accurate TM results

def log_func(x):
    return(np.log10(x + 0.01))

In [33]:
indices_n, angles_n, correlations_n, signs_n = iutls.get_n_best_matches(image, simulations,intensity_transform_function=log_func)

In [34]:
mirrored = signs_n[0] == -1
putls.plot_template_over_pattern(image,
                                 simulations[indices_n[0]],
                                 in_plane_angle=angles_n[0],
                                 size_factor = 1,
                                 vmax=20,
                                 mirrored_template=mirrored,
                                 find_direct_beam=False,
                                 cmap = "inferno"
                                )

(<AxesSubplot:>,
 <matplotlib.collections.PathCollection at 0x7f77bdc81460>)

### Full template matching

In [35]:
result, phasedict = iutls.index_dataset_with_template_rotation(
                                                    work_data,
                                                    diff_lib,
                                                    n_best = 2,
                                                    intensity_transform_function=log_func,
                                                    normalize_images = True,
                                                    )

[########################################] | 100% Completed | 2hr 41m


In [56]:
#Convert python dict to orix CrystalMap object

xmap = iutls.results_dict_to_crystal_map(result,phasedict)

In [57]:
xmap.phases[0].space_group = 185

In [51]:
corrs = xmap.correlation[:,0]
_  = plt.hist(corrs, bins = 100)
plt.show()

In [58]:
xmap.scan_unit = "nm"

In [59]:
from orix import plot
from orix.crystal_map import CrystalMap, Phase, PhaseList
from orix.io import load, save
from orix.quaternion import Orientation, Rotation, symmetry
from orix.vector import Vector3d
import matplotlib.pyplot as plt

In [ ]:
#Initial Orix plot to see that TM result looks OK.

# IMPORTANT: Orientations in this plot will be 30° rotated 
# arond c-axis for ErMnO3 because of different orientation definition (a*||X vs a||X). 
# Could be fixed by multiplying orientations with Rotation.from_axes_angles([0,0,1], np.deg2rad(30))
# MTEX plotting in plot_results.m handles this better.


ckey = plot.IPFColorKeyTSL(xmap.phases["ErMnO3"].point_group, direction=Vector3d.xvector())
Color_X = ckey.orientation2color(xmap["ErMnO3"].orientations)
ckey.direction = Vector3d.yvector()
Color_Y = ckey.orientation2color(xmap["ErMnO3"].orientations)
ckey.direction = Vector3d.zvector()
Color_Z = ckey.orientation2color(xmap["ErMnO3"].orientations)

plt.figure()
fig = plt.figure(figsize=(9,3))
ax0 = fig.add_subplot(131, projection="plot_map")
ax1 = fig.add_subplot(132, projection="plot_map")
ax2 = fig.add_subplot(133, projection="plot_map")
ax0.set_title("X")
ax1.set_title("Y")
ax2.set_title("Z")
ax0.plot_map(xmap["ErMnO3"], Color_X, scalebar=False)
ax1.plot_map(xmap["ErMnO3"], Color_Y, scalebar=False)
ax2.plot_map(xmap["ErMnO3"], Color_Z, scalebar=False)
axes = [ax0,ax1,ax2]
for ax in axes:
    ax.set_xticks([])
    ax.set_yticks([])
fig.tight_layout()

In [36]:
import pickle
def save_result_and_phase_dict(fname, result_dict, phase_dict):
    fnam_result = f"{fname}_result.dict"
    fname_phase = f"{fname}_phase.dict"

    with open(fnam_result, "wb") as filehandler:
        pickle.dump(result_dict, filehandler)

    
    with open(fname_phase, "wb") as filehandler:
        pickle.dump(phase_dict, filehandler)

save_result_and_phase_dict("ErMnO3_SED_left_side", result, phasedict)

In [66]:
#Saves the raw TM xmap, not corrected for scan rotation
# Scan rotation is corrected in Correct_scan_rotation.ipynb
# where xmap is saved as .ang file for further plotting with MTEX

save('ErMnO3_full_lamella.hdf5',xmap) 